# Functions for the database:
1)open_con()->Creates the connection with the database

2)close_con()->Closes the connection with the database

3)tables_info()->Prints the names of the tables from the database,the columns each table contains the type of data

4)df(table)->Returns the specified table as a dataframe

5)db_info()->Generates general information about the database

6)databy_condition(table,conditions,logical_operator)->This function returns the data from a specific table based on one or more conditions applied to multiply columns (or a single column).Example of use:

conditions={'integration_time':(('>','<'),(896,896.9)),
           'central_velocity':('BETWEEN',(11754.1,12234.8))}

df=databy_condition('gbt_observations',conditions,logical_operator='AND')
df
We define the conditions as a dictionary.


7)databy_sort_group_having(df,group_by=None,aggregates=None,having=None,order_by=None,order_type="ASC"):This function returns a grouped or ordered with aggregated data.Example of use:
First we  execute the function df=df(table) or df=databy_condition(table,conditions,logical_operator) if we want data who obey specific conditions.After that we insert the produced dataframe into the function.

df=df('gbt_observations')

databy_sort_group_having(df,group_by='observation_date',aggregates={'rest_frequency':'AVG'})

8)line_plot(table,x_column,y_column)->Produces a line plot

9)heatmap_plot(table,freq_column,time_column,power_column)->Produces a heatmap

10)plot_moving_average(table, freq_column, power_column, window=5)->Produces a line plot using the moving average method for a smoothiest signal

11)heatmap_moving_average(table,freq_column,time_column,power_column,window=5)->Produces a heatmap using the moving average method

12)detect_spectral_lines(table,freq_column,power_column,prominence=0.1)->Detects hydrogen spectral lines

In [1]:
#necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector    #required to execute 'pip install mysql.connector'
import warnings
%matplotlib inline
from graphviz import Digraph

In [2]:
#the next step cteates the connection with the database

In [3]:
db=mysql.connector.connect(
    host="195.201.104.116",
    user="eta14802_astrobeam",
    password="PGdAR{F[1C#~",
    database="eta14802_astrobeam"
)

mycursor=db.cursor()
print(db)

In [4]:
#print the existing tables in the database

In [5]:
mycursor.execute("SHOW TABLES;")
tables=mycursor.fetchall()
print('Database tables:')
for table in tables:
    print(table[0])

Database tables:
gbt_data
gbt_observations
gbt_values


In [6]:
#print the columns of each table

In [7]:
for table in tables:
    table_name = table[0]
    mycursor.execute(f"DESCRIBE {table_name};")
    columns=mycursor.fetchall()
    
    print(f"\nTable columns `{table_name}`:")
    for col in columns:
        print(f"{col[0]} ({col[1]})")



Table columns `gbt_data`:
id (int(11))
vHI (double)
fHI (double)
fBHI (double)
src_file (varchar(100))
telescope (varchar(100))
beam_size (double)
object (varchar(100))
ra_dec (varchar(100))
rest_frequency (double)
central_velocity (double)
integration_time (double)
observation_date (varchar(100))
details (varchar(100))

Table columns `gbt_observations`:
src_file (varchar(100))
telescope (varchar(100))
beam_size (double)
object (varchar(100))
ra_dec (varchar(100))
rest_frequency (double)
central_velocity (double)
integration_time (double)
observation_date (varchar(100))
details (varchar(100))

Table columns `gbt_values`:
id (int(11))
vHI (double)
fHI (double)
fBHI (double)
object (varchar(100))


In [8]:
#print a view of the database(if existing)

In [9]:
mycursor.execute("SHOW FULL TABLES WHERE Table_type = 'VIEW';")
views=mycursor.fetchall()

print("\nViews in database:")
for view in views:
    print(view[0])


Views in database:


In [10]:
#print the relationships in database(Foreign keys to Primary keys)-if existing

In [11]:
mycursor.execute("SELECT TABLE_NAME, COLUMN_NAME, REFERENCED_TABLE_NAME, REFERENCED_COLUMN_NAME FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE WHERE TABLE_SCHEMA = DATABASE()  AND REFERENCED_TABLE_NAME IS NOT NULL;")
    

relations=mycursor.fetchall()

print("Relationships amongst tables (Foreign Keys):")
for rel in relations:
    print(f"- {rel[0]}.{rel[1]} -> {rel[2]}.{rel[3]}")

Relationships amongst tables (Foreign Keys):


In [12]:
#print the primary and foreign keys in each table

In [13]:
mycursor.execute("SELECT TABLE_NAME, COLUMN_NAME FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE WHERE TABLE_SCHEMA = DATABASE() AND CONSTRAINT_NAME = 'PRIMARY';")
primary_keys=mycursor.fetchall()

In [14]:
mycursor.execute("SELECT TABLE_NAME, COLUMN_NAME, REFERENCED_TABLE_NAME, REFERENCED_COLUMN_NAME FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE WHERE TABLE_SCHEMA = DATABASE() AND REFERENCED_TABLE_NAME IS NOT NULL;")
foreign_keys=mycursor.fetchall()

In [15]:
from collections import defaultdict

pk_dict=defaultdict(list)
for table,column in primary_keys:
    pk_dict[table].append(column)

fk_dict=defaultdict(list)
for table,column,ref_table,ref_column in foreign_keys:
    fk_dict[table].append(f"{column} -> {ref_table}.{ref_column}")


print("Primary and Foreign Keys per table:\n")
for table in set(pk_dict.keys()).union(fk_dict.keys()):
    print(f"Table: {table}")
    if table in pk_dict:
        print(f"Primary Keys: {', '.join(pk_dict[table])}")
    if table in fk_dict:
        print(f"Foreign Keys: {', '.join(fk_dict[table])}")
    

Primary and Foreign Keys per table:

Table: gbt_data
Primary Keys: id


In [16]:
#print the constraints 

In [17]:
mycursor.execute("SELECT tc.TABLE_NAME, kc.COLUMN_NAME FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS AS tc JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE AS kc ON tc.CONSTRAINT_NAME = kc.CONSTRAINT_NAME WHERE tc.TABLE_SCHEMA = DATABASE() AND tc.CONSTRAINT_TYPE = 'UNIQUE';")
unique_constraints=mycursor.fetchall()

In [18]:
mycursor.execute("SELECT TABLE_NAME, CONSTRAINT_NAME, CHECK_CLAUSE FROM INFORMATION_SCHEMA.CHECK_CONSTRAINTS WHERE CONSTRAINT_SCHEMA = DATABASE();")
check_constraints=mycursor.fetchall()

In [19]:
unique_dict=defaultdict(list)
for table,column in unique_constraints:
    unique_dict[table].append(column)

check_dict=defaultdict(list)
for table,constraint_name,check_clause in check_constraints:
    check_dict[table].append(f"{constraint_name}: {check_clause}")


print("UNIQUE & CHECK Constraints per Table:\n")
for table in set(unique_dict.keys()).union(check_dict.keys()):
    print(f"Table: {table}")
    if table in unique_dict:
        print(f"UNIQUE Constraints: {', '.join(unique_dict[table])}")
    if table in check_dict:
        print(f"CHECK Constraints: {' | '.join(check_dict[table])}")


UNIQUE & CHECK Constraints per Table:



In [20]:
mycursor.close()
db.close()

In [21]:
def db_info():
    #necessary libraries
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import mysql.connector    #required to execute 'pip install mysql.connector'
    import warnings
    %matplotlib inline
    from graphviz import Digraph
    db=mysql.connector.connect(
    host="195.201.104.116",
    user="eta14802_astrobeam",
    password="PGdAR{F[1C#~",
    database="eta14802_astrobeam"
    )

    mycursor=db.cursor()
    
    mycursor.execute("SHOW TABLES;")
    tables=mycursor.fetchall()
    print('Database tables:')
    for table in tables:
        print(table[0])
        
    print('\n')
    
    
    for table in tables:
        table_name = table[0]
        mycursor.execute(f"DESCRIBE {table_name};")
        columns=mycursor.fetchall()
    
    print(f"\nTable columns `{table_name}`:")
    for col in columns:
        print(f"{col[0]} ({col[1]})")
        
    print('\n')
    
    mycursor.execute("SHOW FULL TABLES WHERE Table_type = 'VIEW';")
    views=mycursor.fetchall()

    print("\nViews in database:")
    for view in views:
        print(view[0])
        
    print('\n')
    
    
    mycursor.execute("SELECT TABLE_NAME, COLUMN_NAME, REFERENCED_TABLE_NAME, REFERENCED_COLUMN_NAME FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE WHERE TABLE_SCHEMA = DATABASE()  AND REFERENCED_TABLE_NAME IS NOT NULL;")
    

    relations=mycursor.fetchall()

    print("Relationships amongst tables (Foreign Keys):")
    for rel in relations:
        print(f"- {rel[0]}.{rel[1]} -> {rel[2]}.{rel[3]}")
        
    print('\n')
    
    mycursor.execute("SELECT TABLE_NAME, COLUMN_NAME FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE WHERE TABLE_SCHEMA = DATABASE() AND CONSTRAINT_NAME = 'PRIMARY';")
    primary_keys=mycursor.fetchall()
    mycursor.execute("SELECT TABLE_NAME, COLUMN_NAME, REFERENCED_TABLE_NAME, REFERENCED_COLUMN_NAME FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE WHERE TABLE_SCHEMA = DATABASE() AND REFERENCED_TABLE_NAME IS NOT NULL;")
    foreign_keys=mycursor.fetchall()
    from collections import defaultdict

    pk_dict=defaultdict(list)
    for table,column in primary_keys:
        pk_dict[table].append(column)

    fk_dict=defaultdict(list)
    for table,column,ref_table,ref_column in foreign_keys:
        fk_dict[table].append(f"{column} -> {ref_table}.{ref_column}")


    print("Primary and Foreign Keys per table:\n")
    for table in set(pk_dict.keys()).union(fk_dict.keys()):
        print(f"Table: {table}")
        if table in pk_dict:
            print(f"Primary Keys: {', '.join(pk_dict[table])}")
        if table in fk_dict:
            print(f"Foreign Keys: {', '.join(fk_dict[table])}")
            
    print('n')
    
    mycursor.execute("SELECT tc.TABLE_NAME, kc.COLUMN_NAME FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS AS tc JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE AS kc ON tc.CONSTRAINT_NAME = kc.CONSTRAINT_NAME WHERE tc.TABLE_SCHEMA = DATABASE() AND tc.CONSTRAINT_TYPE = 'UNIQUE';")
    unique_constraints=mycursor.fetchall()
    mycursor.execute("SELECT TABLE_NAME, CONSTRAINT_NAME, CHECK_CLAUSE FROM INFORMATION_SCHEMA.CHECK_CONSTRAINTS WHERE CONSTRAINT_SCHEMA = DATABASE();")
    check_constraints=mycursor.fetchall()
    unique_dict=defaultdict(list)
    for table,column in unique_constraints:
        unique_dict[table].append(column)

    check_dict=defaultdict(list)
    for table,constraint_name,check_clause in check_constraints:
        check_dict[table].append(f"{constraint_name}: {check_clause}")


    print("UNIQUE & CHECK Constraints per Table:\n")
    for table in set(unique_dict.keys()).union(check_dict.keys()):
        print(f"Table: {table}")
        if table in unique_dict:
            print(f"UNIQUE Constraints: {', '.join(unique_dict[table])}")
        if table in check_dict:
            print(f"CHECK Constraints: {' | '.join(check_dict[table])}")
            
            
    mycursor.close()
    db.close()

    

In [22]:
def open_con():
    import mysql.connector
    db=mysql.connector.connect(
    host="195.201.104.116",
    user="eta14802_astrobeam",
    password="PGdAR{F[1C#~",
    database="eta14802_astrobeam"
    )

    mycursor=db.cursor()
    print(db)

In [23]:
def close_con():
    mycursor.close()
    db.close()

In [24]:
def tables_info():
    import mysql.connector
    db=mysql.connector.connect(
    host="195.201.104.116",
    user="eta14802_astrobeam",
    password="PGdAR{F[1C#~",
    database="eta14802_astrobeam"
    )

    mycursor=db.cursor()
    
    
    for table in tables:
        table_name = table[0]
        mycursor.execute(f"DESCRIBE {table_name};")
        columns=mycursor.fetchall()
    
        print(f"\nTable columns `{table_name}`:")
        for col in columns:
            print(f"{col[0]} ({col[1]})")

def databy_condition(table,column,operator,value):
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import mysql.connector   
    import warnings
    %matplotlib inline
    
    try:
        db=mysql.connector.connect(
        host="195.201.104.116",
        user="eta14802_astrobeam",
        password="PGdAR{F[1C#~",
        database="eta14802_astrobeam"
        )
        mycursor=db.cursor()
        
        
        if operator=="BETWEEN" and isinstance(value,tuple):
            query = f"SELECT * FROM {table} WHERE {column} BETWEEN {value[0]} AND {value[1]}"
            mycursor.execute(query)
        else:
            query=f"SELECT * FROM {table} WHERE {column} {operator} {value}"
            mycursor.execute(query)

        
        result=mycursor.fetchall()

        
        columns=[desc[0] for desc in mycursor.description]  
        df = pd.DataFrame(result, columns=columns)

        return df  

    except mysql.connector.Error as err:
        print(f'Error: {err}')
        return None

    finally:
        mycursor.close()
        db.close()
        
    
    

In [26]:
def databy_condition(table, conditions, logical_operator="AND"):
    import numpy as np
    import pandas as pd
    import mysql.connector
    

    db = mysql.connector.connect(
        host="195.201.104.116",
        user="eta14802_astrobeam",
        password="PGdAR{F[1C#~",
        database="eta14802_astrobeam"
    )
    
    mycursor = db.cursor()
    

    condition_statements = []
    

    for column, (operator, value) in conditions.items():
        if isinstance(operator, tuple) and isinstance(value, tuple) and len(operator) == 2:
        
            condition_statements.append(f"{column} {operator[0]} {value[0]} AND {column} {operator[1]} {value[1]}")
        elif operator == "BETWEEN" and isinstance(value, tuple):
            condition_statements.append(f"{column} BETWEEN {value[0]} AND {value[1]}")
        elif operator == "IN" and isinstance(value, (list, tuple)):
            value_list = ", ".join(map(str, value))
            condition_statements.append(f"{column} IN ({value_list})")
        elif operator == "LIKE":
            condition_statements.append(f"{column} LIKE '{value}'")
        else:
            condition_statements.append(f"{column} {operator} {value}")
    
    where_clause = f" {logical_operator} ".join(condition_statements)
    

    query = f"SELECT * FROM {table} WHERE {where_clause};"
    
    mycursor.execute(query)
    
    result = mycursor.fetchall()
    
    columns = [desc[0] for desc in mycursor.description]
    df = pd.DataFrame(result, columns=columns)
    

    mycursor.close()
    db.close()
    
    return df

In [27]:
conditions={'integration_time':(('>','<'),(896,896.9)),
           'central_velocity':('BETWEEN',(11754.1,12234.8))}
df=databy_condition('gbt_observations',conditions,logical_operator='AND')
df
                               

,src_file,telescope,beam_size,object,ra_dec,rest_frequency,central_velocity,integration_time,observation_date,details
0,mangaHI-10001-12701.csv,Robert C. Bryd Green Bank Telescope,9.0,10001-12701,08 53 29.1 +57 35 54,1420.4058,11754.1,896.8,2017-01-16,N. Samanso Tue Jan 21 11:00:44 2020
1,mangaHI-10001-12702.csv,Robert C. Bryd Green Bank Telescope,9.0,10001-12702,08 54 44.6 +57 28 49,1420.4058,12234.8,896.8,2017-01-17,N. Samanso Tue Jan 21 11:07:58 2020
2,mangaHI-10001-3701.csv,Robert C. Bryd Green Bank Telescope,9.0,10001-3701,08 49 51.8 +57 08 37,1420.4058,11817.5,896.8,2017-01-16,N. Samanso Tue Jan 21 11:02:34 2020
3,mangaHI-10001-3703.csv,Robert C. Bryd Green Bank Telescope,9.0,10001-3703,08 58 22.0 +57 41 06,1420.4058,12157.4,896.8,2017-01-17,N. Samanso Fri Jan 24 15:25:09 2020
4,mangaHI-10001-6103.csv,Robert C. Bryd Green Bank Telescope,9.0,10001-6103,08 56 01.9 +57 23 27,1420.4058,12179.5,896.8,2017-01-17,N. Samanso Thu Jan 23 16:08:09 2020
5,mangaHI-10215-1902.csv,Robert C. Bryd Green Bank Telescope,9.0,10215-1902,08 15 25.7 +37 20 27,1420.4058,11910.9,896.5,2019-03-05,Karen Masters Sat Feb 13 15:49:54 2021
6,mangaHI-10215-6103.csv,Robert C. Bryd Green Bank Telescope,9.0,10215-6103,08 09 17.6 +38 20 13,1420.4058,12206.1,896.4,2019-03-05,Karen L. Masters Thu Apr 22 15:09:43 2021
7,mangaHI-10224-3701.csv,Robert C. Bryd Green Bank Telescope,9.0,10224-3701,02 17 35.8 -00 29 37,1420.4058,12177.8,896.6,2021-01-06,D. V. Stark Mon Sep 27 16:00:04 2021
8,mangaHI-10492-6101.csv,Robert C. Bryd Green Bank Telescope,9.0,10492-6101,08 14 52.2 +55 36 38,1420.4058,12118.5,896.1,2019-01-07,Anubhav Sharma Tue Dec 31 17:22:18 2019
9,mangaHI-10494-3704.csv,Robert C. Bryd Green Bank Telescope,9.0,10494-3704,08 25 23.9 +54 50 22,1420.4058,11852.9,896.3,2019-03-25,Karen Masters Mon Mar 15 17:02:44 2021


In [28]:
def df(table):
    import numpy as np
    import pandas as pd
    import mysql.connector
    db=mysql.connector.connect(
    host="195.201.104.116",
    user="eta14802_astrobeam",
    password="PGdAR{F[1C#~",
    database="eta14802_astrobeam"
    )

    mycursor=db.cursor()
    query = f"SELECT * FROM {table}"
    mycursor.execute(query)
    
    result = mycursor.fetchall()
    columns = [desc[0] for desc in mycursor.description]
    df = pd.DataFrame(result, columns=columns)

    mycursor.close()
    db.close()

    return df

In [29]:
def databy_sort_group_having(df,group_by=None,aggregates=None,having=None,order_by=None,order_type="ASC"):
    
    

    if group_by and aggregates:
        if isinstance(group_by, str):
            group_by = [group_by] 
        
        
        agg_funcs = {
            "SUM": "sum",
            "AVG": "mean",
            "COUNT": "count",
            "MIN": "min",
            "MAX": "max"
        }
        
    
        agg_dict = {col: agg_funcs[func] for col, func in aggregates.items() if func in agg_funcs}
        
        df = df.groupby(group_by).agg(agg_dict).reset_index()

    if having:
        for column, (operator, value) in having.items():
            if operator == ">":
                df = df[df[column] > value]
            elif operator == "<":
                df = df[df[column] < value]
            elif operator == ">=":
                df = df[df[column] >= value]
            elif operator == "<=":
                df = df[df[column] <= value]
            elif operator == "==":
                df = df[df[column] == value]
            elif operator == "!=":
                df = df[df[column] != value]

    
    if order_by:
        ascending = True if order_type.upper() == "ASC" else False
        df = df.sort_values(by=order_by, ascending=ascending)

    return df

In [30]:
df=df('gbt_observations')

In [31]:
databy_sort_group_having(df,group_by='observation_date',aggregates={'rest_frequency':'AVG'})

,observation_date,rest_frequency
0,2016-02-05,1420.4058
1,2016-02-08,1420.4058
2,2016-02-09,1420.4058
3,2016-02-10,1420.4058
4,2016-02-11,1420.4058
...,...,...
497,2021-05-14,1420.4058
498,2021-05-15,1420.4058
499,2021-05-16,1420.4058
500,2021-05-17,1420.4058


In [32]:
def line_plot(table,x_column,y_column):
    
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import mysql.connector
    
    
    db=mysql.connector.connect(
        host="195.201.104.116",
        user="eta14802_astrobeam",
        password="PGdAR{F[1C#~",
        database="eta14802_astrobeam"
    )
    mycursor = db.cursor()
    
    query = f"SELECT {x_column},{y_column} FROM {table}"
    mycursor.execute(query)
    
    result = mycursor.fetchall()
    columns = [desc[0] for desc in mycursor.description]
    df=pd.DataFrame(result,columns=columns)
    
    x=np.array(df[x_column])
    y=np.array(df[y_column])
    
    mycursor.close()
    db.close()

    plt.figure(figsize=(12,6))
    plt.plot(x,y,color="blue",linestyle="-")
    plt.xlabel("Frequency (MHz)")
    plt.ylabel("Power")
    plt.title("Radio Spectrum")
    plt.show()

    return df 

In [33]:
def heatmap_plot(table,freq_column,time_column,power_column):
    
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import mysql.connector
    
    

    db = mysql.connector.connect(
        host="195.201.104.116",
        user="eta14802_astrobeam",
        password="PGdAR{F[1C#~",
        database="eta14802_astrobeam"
    )
    mycursor = db.cursor()

    query=f"SELECT {freq_column}, {time_column}, {power_column} FROM {table}"
    mycursor.execute(query)
    result=mycursor.fetchall()
    columns=[desc[0] for desc in mycursor.description]
    df=pd.DataFrame(result,columns=columns)

    mycursor.close()
    db.close()

    freq_array=df[freq_column].to_numpy()
    time_array=df[time_column].to_numpy()
    power_array=df[power_column].to_numpy()

    heatmap_data=df.pivot_table(index=time_column,columns=freq_column,values=power_column,aggfunc='mean')

    plt.figure(figsize=(12, 6))
    ax = sns.heatmap(heatmap_data,cmap="inferno",cbar=True)

    if (freq_array==1420).any():
        idx_1420=np.abs(np.unique(freq_array) - 1420).argmin()  
        plt.axvline(x=idx_1420,color="red",linestyle="--",linewidth=2,label="Hydrogen Line (1420 MHz)")

    plt.xlabel("Frequency (MHz)")
    plt.ylabel("Time")
    plt.title("Radio Spectrum Heatmap")
    plt.legend()
    plt.show()

    return df

In [34]:
def plot_moving_average(table,freq_column,power_column,window=5):
    
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import mysql.connector
    
    db=mysql.connector.connect(
        host="195.201.104.116",
        user="eta14802_astrobeam",
        password="PGdAR{F[1C#~",
        database="eta14802_astrobeam"
    )
    mycursor = db.cursor()

    query=f"SELECT {freq_column}, {power_column} FROM {table} ORDER BY {freq_column}"
    mycursor.execute(query)
    result=mycursor.fetchall()
    columns=[desc[0] for desc in mycursor.description]
    df=pd.DataFrame(result,columns=columns)

    mycursor.close()
    db.close()

    freq_array=df[freq_column].to_numpy()
    power_array=df[power_column].to_numpy()

    smooth_power=np.convolve(power_array, np.ones(window)/window, mode='valid')

    smooth_freq=freq_array[:len(smooth_power)]


    plt.figure(figsize=(12, 6))
    plt.plot(freq_array,power_array,alpha=0.4,label="Raw Signal",color="gray")
    plt.plot(smooth_freq,smooth_power,label=f"Moving Average (window={window})",color="red")

    if (1420 in freq_array):
        plt.axvline(x=1420,color="blue",linestyle="--",linewidth=2,label="Hydrogen Line (1420 MHz)")

    plt.xlabel("Frequency (MHz)")
    plt.ylabel("Power")
    plt.title("Smoothed Radio Spectrum (Moving Average)")
    plt.legend()
    plt.show()

    return df

In [35]:
def heatmap_moving_average(table,freq_column,time_column,power_column,window=5):

    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import mysql.connector
    
    
    db = mysql.connector.connect(
        host="195.201.104.116",
        user="eta14802_astrobeam",
        password="PGdAR{F[1C#~",
        database="eta14802_astrobeam"
    )
    mycursor = db.cursor()

    query=f"SELECT {freq_column}, {time_column}, {power_column} FROM {table}"
    mycursor.execute(query)
    result=mycursor.fetchall()
    columns=[desc[0] for desc in mycursor.description]
    df=pd.DataFrame(result, columns=columns)

    mycursor.close()
    db.close()

    df=df.sort_values(by=[time_column, freq_column])

    freq_array=np.array(df[freq_column])
    time_array=np.array(df[time_column])
    power_array=np.array(df[power_column])

    smoothed_power=np.convolve(power_array,np.ones(window)/window,mode='same')

    df[power_column]=smoothed_power  
    heatmap_data=df.pivot_table(index=time_column,columns=freq_column,values=power_column,aggfunc='mean')


    plt.figure(figsize=(12, 6))
    ax=sns.heatmap(heatmap_data,cmap="inferno",cbar=True)

    if (freq_array==1420).any():
        idx_1420=np.abs(np.unique(freq_array) - 1420).argmin()  
        ax.axvline(x=idx_1420,color="cyan",linestyle="-",linewidth=2,label="Hydrogen Line (1420 MHz)")

    plt.xlabel("Frequency (MHz)")
    plt.ylabel("Time")
    plt.title("Smoothed Radio Heatmap")
    plt.legend()
    plt.show()

    return df

In [194]:
def detect_spectral_lines(table,freq_column,power_column,prominence=0.1):
    
    
    import mysql.connector
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.signal import find_peaks
   

    db=mysql.connector.connect(
        host="195.201.104.116",
        user="eta14802_astrobeam",
        password="PGdAR{F[1C#~",
        database="eta14802_astrobeam"
    )
    mycursor=db.cursor()

    query=f"SELECT {freq_column}, {power_column} FROM {table}"
    mycursor.execute(query)
    result=mycursor.fetchall()
    columns=[desc[0] for desc in mycursor.description]
    df=pd.DataFrame(result, columns=columns)

    mycursor.close()
    db.close()

    freq_array=np.array(df[freq_column])
    power_array=np.array(df[power_column])

    peaks, _=find_peaks(power_array, prominence=prominence)

    troughs, _=find_peaks(-power_array, prominence=prominence)

    plt.figure(figsize=(12,6))
    plt.plot(freq_array,power_array,label="Spectrum",color="black")
    plt.scatter(freq_array[peaks],power_array[peaks],color="red",label="Emission Lines",marker="o", s=80)
    plt.scatter(freq_array[troughs],power_array[troughs],color="blue",label="Absorption Lines",marker="x", s=80)

    if 1420 in freq_array:
        plt.axvline(x=1420,color="cyan",linestyle="--",linewidth=2,label="Hydrogen Line (1420 MHz)")

    plt.xlabel("Frequency (MHz)")
    plt.ylabel("Power")
    plt.title("Detected Spectral Lines")
    plt.legend()
    plt.show()

    detected_lines = {
        "emission": freq_array[peaks],
        "absorption": freq_array[troughs]
    }

    return detected_lines, df